# check gene list match LG3 , LG4 (LG3 + AF filter) : for jsuthusky  ,  KI pathogenic, and remaining half 50%(justhusky+KI)

1. LG4: if use AF filter , then add function `def filter_AF`; 
2. LG3: if not, only see the LG3 pred, then comment the function 

Method:  
* check if genelist' name match our datasets' Symbol
1. for justhusky: check how many prediction pathogenic by LG matched genelists
2. for newpatho: just check how many newpatho orignial file matched genelist


In [2]:
import gzip
import pandas as pd
import numpy as np
from joblib import dump, load
from sklearn.compose import make_column_transformer
from sklearn.impute import MissingIndicator, SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from category_encoders import BinaryEncoder

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt

/Users/nancy/miniconda3/envs/bioinfo/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# 1. justhusky

In [3]:
""" FIRST of ALL-- user need"""

##### please set files
# in
# vcfgz_in_file = '/Users/nancy/Desktop/RS_projects/data/03_ML/03_predict_new/01_featureV1/justhusky_gatkcomb_rhocall_norm_af_mt_frqf_cadd_vep_parsed_ranked.vcf.gz'
variant_consequences_order = './variant_consequences.txt'

# out
# pred_out_simple = '/Users/nancy/Desktop/RS_projects/result/04_ML_modify/02_evaluate/01_Clinvar_pred-29feats_CJP5-5_LG.csv'


##### load fitted preprocessor and model
preprocessor = load('/Users/nancy/Desktop/RS_projects/result/04_ML_modify/01_retrainModel_ClinvarJusthuskyNewpatho/models/02_12feats_ClinJustPatho5-5_preprocessor.joblib') # data preprocessor
LG_model = load('/Users/nancy/Desktop/RS_projects/result/04_ML_modify/01_retrainModel_ClinvarJusthuskyNewpatho/models/02_12feats_ClinJustPatho5-5_LG.joblib')         # logistic regression model


In [4]:
## predict


df=pd.read_csv('/Users/nancy/Desktop/RS_projects/result/03_ML/03_predict_new/01_featureV1/01_justhusky_features_noclnsig.csv',sep=';')
df['idx'] = df.index  # add index column

""" 2.1 choose featuresV1"""
### actually, no need change to 12 features here, since our preprocessor will only transfer and get 12 features as well, so doesn't matter if you change here or not

featureV1 = ['idx','CADD', 'AF_TGP', 'Frq', 'IMPACT','GNOMADAF_popmax', 'Hom', 'ORIGIN',
       'SPIDEX', 'SWEGENAF', 'Consequence', 'BIOTYPE', 'SIFT',
       'PolyPhen', 'MES-SWA_acceptor_alt', 'MES-SWA_acceptor_diff',
       'MES-SWA_donor_alt', 'MES-SWA_donor_diff', 'MaxEntScan_alt',
       'MaxEntScan_diff', 'GERP++_RS', 'REVEL_score',
       'phastCons100way_vertebrate', 'phyloP100way_vertebrate', 'LoFtool',
       'pLI_gene_value', 'SpliceAI_pred_DS_AG', 'SpliceAI_pred_DS_AL',
       'SpliceAI_pred_DS_DG', 'SpliceAI_pred_DS_DL']         # remove CLNSIG,   add index: 29+1 total

df_1 = df[featureV1]



""" 2.2 preprocessing and predict """

## predict function :
def predict_score(X,model,df_ori):
    # Use preprocessor and model to predict on new data
    X_processed = preprocessor.transform(X)
    pred = model.predict(X_processed)
    prob = model.predict_proba(X_processed)

    # put prediction and score back to dataframe with same index
    df_out = pd.DataFrame(data = [pred,prob[:,1]]).T
    df_out.columns = ['prediction','score']
    df_out['score']=df_out['score'].round(3)
    df_out = pd.concat([X['idx'],df_out],axis=1)

    #  write model outputs in original df at same index rows
    df_copy = df_ori.copy() 
    df_merged = pd.merge(df_ori,df_out, on='idx')    # get only rows merged when index is same
    df_copy.loc[df_copy['idx'].isin(df_merged['idx']),['prediction','score']] = df_merged[['prediction','score']].values  # put cells of merged in original dataframe
    lst = ['idx','prediction','score']
    return df_copy[lst]

# data preprocessing and predicted by models
df_lg = predict_score(df_1,LG_model,df)

# df_lg.to_csv(pred_out_simple,index=False)




/var/folders/q0/2p8kfxsn7s73349btf1_g07h0000gp/T/ipykernel_97101/2892436993.py:4: DtypeWarning: Columns (22,23,24,25,26,27,31,37,39,40,41,48,49,50,51,53,55,57,66,90,92,94,95) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('/Users/nancy/Desktop/RS_projects/result/03_ML/03_predict_new/01_featureV1/01_justhusky_features_noclnsig.csv',sep=';')


In [5]:
""" function: filter prediction by AF > 0.01:
if any AF > 0.01 & patho, change patho to benign 

output: precdiction by model after filtering """


# function:  to get the minimum non-NaN value in a row
def min_nonnan(row):
    nonnan_vals = [val for val in row if not np.isnan(val)]
    return min(nonnan_vals) if nonnan_vals else np.nan


## function: transfer `predicted patho`` with any AF>=0.01, to `benign`
def filter_AF(df_extracted_ori,df_pred_out):

    af = ['AF_ESP', 'AF_EXAC', 'AF_TGP', 'Frq', 'GNOMADAF_popmax', 'SWEGENAF']
    df_af = df_extracted_ori[af]  # Extract all related to AF parameters from the data

    df_merge_af = pd.concat([df_pred_out, df_af], axis=1)
    df_merge_af['AF_min'] = df_merge_af[af].apply(min_nonnan, axis=1)   # create a new 'AF_min' column

    
    df_filter = df_merge_af.copy()
    df_filter.loc[(df_filter['prediction'] == 1) & (df_filter['AF_min'] >= 0.01), ['prediction']] = 0   # Change 'patho' to 'benign' when min 'AF' is >= 0.01

    # Extract the 'prediction' and 'score' columns from the filtered data
    df_pred_out_1 = df_filter[['prediction', 'score']]

    # Return the value counts of 'prediction'
    return df_pred_out_1

df_lg = filter_AF(df,df_lg)

## evaluate   Justhusy - prediction results: for all Justhusky variants

In [6]:
df_lg.prediction.value_counts()

0.0    4575006
1.0        268
Name: prediction, dtype: int64

## merge symbol

In [7]:
# merge all dataframes
df_merge_just = pd.concat([df_lg,df.SYMBOL],axis=1)
df_merge_just 

,prediction,score,SYMBOL
0,0.0,0.0,DDX11L1
1,0.0,0.0,DDX11L1
2,0.0,0.0,DDX11L1
3,0.0,0.0,DDX11L1
4,0.0,0.0,DDX11L1
...,...,...,...
4575269,0.0,0.0,NaN
4575270,0.0,0.0,NaN
4575271,0.0,0.0,NaN
4575272,0.0,0.0,NaN


In [13]:
## only get pred pathogenic
df_merge_just_predpatho = df_merge_just .loc[df_merge_just ['prediction'].isin([1])]
df_merge_just_predpatho

,prediction,score,SYMBOL
4652,1.0,0.857,PRDM16
10368,1.0,0.818,NPHP4
19763,1.0,0.950,VPS13D
20341,1.0,0.651,PRAMEF11
67833,1.0,0.815,SLC6A9
...,...,...,...
4396283,1.0,0.957,AC002472.1
4426131,1.0,0.521,NPTXR
4499162,1.0,0.741,ASB12
4563194,1.0,0.831,HMGB3


## genelist check match:
for justhusky healthy person, check how many perdicted pathogenic by LG model, in genelist

In [12]:
## get 3 gene list: 

df_INTE = pd.read_csv('/Users/nancy/Desktop/RS_projects/data/03_ML/gene_list/Intellectual_disability_microarray_and_sequencing.tsv',sep='\t')
df_INTE

# count how many `entity name`` is same to `gene symbl :
count_same = (df_INTE['Entity Name'] == df_INTE['Gene Symbol']).sum()
count_same

df_INTE = df_INTE['Entity Name']



df_PRI = pd.read_csv('/Users/nancy/Desktop/RS_projects/data/03_ML/gene_list/Primary_ciliary_disorders.tsv',sep='\t')
df_PRI

# count how many `entity name`` is same to `gene symbl
count_same = (df_PRI['Entity Name'] == df_PRI['Gene Symbol']).sum()
count_same

df_PRI = df_PRI['Entity Name']



df_STR = pd.read_csv('/Users/nancy/Desktop/RS_projects/data/03_ML/gene_list/Structural_eye_disease.tsv',sep='\t')
df_STR

# count how many `entity name`` is same to `gene symbl
count_same = (df_STR['Entity Name'] == df_STR['Gene Symbol']).sum()
count_same

df_STR = df_STR['Entity Name']




### counts: match `symbol` in pathogenic prediction to `entity name` in gene list
in gene list: most Entity name is same to Gene Symbol, but some Symbol is Nan since the name is region instead of gene

In [14]:
## LG  list_INTE :  how many pathogenic prediction's 'Symbol' matched in the gene lists' 'Entity Name'

# Merging the dataframes based on the columns 'a' and 'b'
match_result_LG_INT = pd.merge(df_merge_just_predpatho, df_INTE, left_on='SYMBOL', right_on='Entity Name', how='inner')

# Printing the merge result and count of matches
match_result_LG_INT
print("Number of Matches: ", len(match_result_LG_INT))

#### count the unique values in matching:
unique_values = match_result_LG_INT['SYMBOL'].unique()
print('the number of unique matching:',len(unique_values))

Number of Matches:  38
the number of unique matching: 37


In [15]:
## LG  df_PRI :  how many pathogenic prediction's 'Symbol' matched in the gene lists' 'Entity Name'

# Merging the dataframes based on the columns 'a' and 'b'
match_result_LG_PRI = pd.merge(df_merge_just_predpatho, df_PRI, left_on='SYMBOL', right_on='Entity Name', how='inner')

# Printing the merge result and count of matches
match_result_LG_PRI
print("Number of Matches: ", len(match_result_LG_PRI))

unique_values = match_result_LG_PRI['SYMBOL'].unique()
print('the number of unique matching:',len(unique_values))

Number of Matches:  5
the number of unique matching: 5


In [16]:
## LG  df_STR :  how many pathogenic prediction's 'Symbol' matched in the gene lists' 'Entity Name'

# Merging the dataframes based on the columns 'a' and 'b'
match_result_LG_STR = pd.merge(df_merge_just_predpatho, df_STR, left_on='SYMBOL', right_on='Entity Name', how='inner')

# Printing the merge result and count of matches
match_result_LG_STR
print("Number of Matches: ", len(match_result_LG_STR))

unique_values = match_result_LG_STR['SYMBOL'].unique()
print('the number of unique matching:',len(unique_values))

Number of Matches:  10
the number of unique matching: 10


# 2. new patho

In [8]:
## predict


df=pd.read_csv('/Users/nancy/Desktop/RS_projects/result/03_ML/03_predict_new/02_featureV2/015_NEWpatho_features_noclnsig.csv',sep=';')
df['idx'] = df.index  # add index column

""" 2.1 choose featuresV1"""

featureV1 = ['idx','CADD', 'AF_TGP', 'Frq', 'IMPACT','GNOMADAF_popmax', 'Hom', 'ORIGIN',
       'SPIDEX', 'SWEGENAF', 'Consequence', 'BIOTYPE', 'SIFT',
       'PolyPhen', 'MES-SWA_acceptor_alt', 'MES-SWA_acceptor_diff',
       'MES-SWA_donor_alt', 'MES-SWA_donor_diff', 'MaxEntScan_alt',
       'MaxEntScan_diff', 'GERP++_RS', 'REVEL_score',
       'phastCons100way_vertebrate', 'phyloP100way_vertebrate', 'LoFtool',
       'pLI_gene_value', 'SpliceAI_pred_DS_AG', 'SpliceAI_pred_DS_AL',
       'SpliceAI_pred_DS_DG', 'SpliceAI_pred_DS_DL']         # remove CLNSIG,   add index: 29+1 total

df_1 = df[featureV1]



""" 2.2 preprocessing and predict """

## predict function :
def predict_score(X,model,df_ori):
    # Use preprocessor and model to predict on new data
    X_processed = preprocessor.transform(X)
    pred = model.predict(X_processed)
    prob = model.predict_proba(X_processed)

    # put prediction and score back to dataframe with same index
    df_out = pd.DataFrame(data = [pred,prob[:,1]]).T
    df_out.columns = ['prediction','score']
    df_out['score']=df_out['score'].round(3)
    df_out = pd.concat([X['idx'],df_out],axis=1)

    #  write model outputs in original df at same index rows
    df_copy = df_ori.copy() 
    df_merged = pd.merge(df_ori,df_out, on='idx')    # get only rows merged when index is same
    df_copy.loc[df_copy['idx'].isin(df_merged['idx']),['prediction','score']] = df_merged[['prediction','score']].values  # put cells of merged in original dataframe
    lst = ['idx','prediction','score']
    return df_copy[lst]

# data preprocessing and predicted by models
df_lg_newpatho = predict_score(df_1,LG_model,df)

# df_lg.to_csv(pred_out_simple,index=False)

In [9]:
df_lg_newpatho = filter_AF(df,df_lg_newpatho)

### evaluate all new patho

In [44]:
df_lg_newpatho

,prediction,score
0,1.0,0.717
1,0.0,0.367
2,0.0,0.053
3,0.0,0.116
4,1.0,0.972
...,...,...
972,1.0,0.958
973,1.0,0.979
974,1.0,0.841
975,1.0,0.760


In [10]:
df_lg_newpatho.prediction.value_counts()

1.0    789
0.0    188
Name: prediction, dtype: int64

## genelist check match: all new patho
df are all patho

### check all new patho match in genelist

In [17]:
df_allsymbol = df.SYMBOL


In [18]:
## all patho  list_INTE :  how many pathogenic prediction's 'Symbol' matched in the gene lists' 'Entity Name'

# Merging the dataframes based on the columns 'a' and 'b'
match_result_LG_INT = pd.merge(df_allsymbol, df_INTE, left_on='SYMBOL', right_on='Entity Name', how='inner')

# Printing the merge result and count of matches
match_result_LG_INT
print("Number of Matches: ", len(match_result_LG_INT))

#### count the unique values in matching:
unique_values = match_result_LG_INT['SYMBOL'].unique()
print('the number of unique matching:',len(unique_values))

Number of Matches:  718
the number of unique matching: 422


In [19]:
## all patho df_PRI :  how many pathogenic prediction's 'Symbol' matched in the gene lists' 'Entity Name'

# Merging the dataframes based on the columns 'a' and 'b'
match_result_LG_PRI = pd.merge(df_allsymbol, df_PRI, left_on='SYMBOL', right_on='Entity Name', how='inner')

# Printing the merge result and count of matches
match_result_LG_PRI
print("Number of Matches: ", len(match_result_LG_PRI))

unique_values = match_result_LG_PRI['SYMBOL'].unique()
print('the number of unique matching:',len(unique_values))

Number of Matches:  31
the number of unique matching: 21


In [20]:
## all patho  df_STR :  how many pathogenic prediction's 'Symbol' matched in the gene lists' 'Entity Name'

# Merging the dataframes based on the columns 'a' and 'b'
match_result_LG_STR = pd.merge(df_allsymbol, df_STR, left_on='SYMBOL', right_on='Entity Name', how='inner')

# Printing the merge result and count of matches
match_result_LG_STR
print("Number of Matches: ", len(match_result_LG_STR))

unique_values = match_result_LG_STR['SYMBOL'].unique()
print('the number of unique matching:',len(unique_values))

Number of Matches:  123
the number of unique matching: 70


# 3. half (just + newpatho)  test data:
forget to add AF filter, not matter ,just see for fun 

In [25]:
## get half just + newpatho test data including rankscore:

test_half_just = pd.read_csv('/Users/nancy/Desktop/RS_projects/data/04_ML_modify/01_retrainModel_ClinvarJusthuskyNewpatho/01_test_half_justhusky.csv')
test_half_newpatho = pd.read_csv('/Users/nancy/Desktop/RS_projects/data/04_ML_modify/01_retrainModel_ClinvarJusthuskyNewpatho/01_test_half_newpatho.csv')


/var/folders/q0/2p8kfxsn7s73349btf1_g07h0000gp/T/ipykernel_97101/262486184.py:3: DtypeWarning: Columns (22,24,25,26,27,31,48,49,50,51,66,92,94,95) have mixed types. Specify dtype option on import or set low_memory=False.
  test_half_just = pd.read_csv('/Users/nancy/Desktop/RS_projects/data/04_ML_modify/01_retrainModel_ClinvarJusthuskyNewpatho/01_test_half_justhusky.csv')


In [ ]:
## get half just + newpatho test data including rankscore:

merge_test_half_addrankscore = pd.read_csv('/Users/nancy/Desktop/RS_projects/result/04_ML_modify/01_retrainModel_ClinvarJusthuskyNewpatho/01_extract_oldRankscore_29feats_testhalf_JustNewpatho.csv')

In [24]:
""" ##  2. Data preprocessing and prediction ## """


""" 2.1 choose featuresV1"""

merge_test_half_addrankscore['idx'] = merge_test_half_addrankscore.index  # add index column


featureV1 = ['idx',
'CADD',
'Frq',
'GNOMADAF_popmax',
'Consequence',
'BIOTYPE',
'PolyPhen',
'REVEL_score',
'pLI_gene_value',
'SpliceAI_pred_DS_AG',
'SpliceAI_pred_DS_AL',
'SpliceAI_pred_DS_DG',
'SpliceAI_pred_DS_DL']         # 12


df_1 = merge_test_half_addrankscore[featureV1]



""" 2.2 preprocessing and predict """

## predict function :
def predict_score(X,model,df_ori):
    # Use preprocessor and model to predict on new data
    X_processed = preprocessor.transform(X)
    pred = model.predict(X_processed)
    prob = model.predict_proba(X_processed)

    # put prediction and score back to dataframe with same index
    df_out = pd.DataFrame(data = [pred,prob[:,1]]).T
    df_out.columns = ['prediction','score']
    df_out['score']=df_out['score'].round(3)
    df_out = pd.concat([X['idx'],df_out],axis=1)

    #  write model outputs in original df at same index rows
    df_copy = df_ori.copy() 
    df_merged = pd.merge(df_ori,df_out, on='idx')    # get only rows merged when index is same
    df_copy.loc[df_copy['idx'].isin(df_merged['idx']),['prediction','score']] = df_merged[['prediction','score']].values  # put cells of merged in original dataframe
    lst = ['idx','prediction','score']
    return df_copy[lst]

# data preprocessing and predicted by models
df_lg = predict_score(df_1,LG_model,merge_test_half_addrankscore)


# df_lg.to_csv(pred_out_simple,index=False)


In [48]:
df_lg

,idx,prediction,score
0,0,0.0,0.000
1,1,0.0,0.000
2,2,0.0,0.000
3,3,0.0,0.000
4,4,0.0,0.000
...,...,...,...
2288121,2288121,1.0,0.992
2288122,2288122,1.0,0.993
2288123,2288123,1.0,0.841
2288124,2288124,1.0,0.760


### merge: pred + symbol

In [31]:
## NOTE: order -- just up, newpatho down

df_half_merge_symble= pd.concat((test_half_just.SYMBOL,test_half_newpatho.SYMBOL),axis = 0)
df_half_merge_symble = df_half_merge_symble.reset_index(drop=True)

In [57]:
df_half_merge= pd.concat((df_lg,df_half_merge_symble),axis = 1)
df_half_merge

,prediction,score,SYMBOL
0,0.0,0.000,DDX11L1
1,0.0,0.000,DDX11L1
2,0.0,0.000,DDX11L1
3,0.0,0.000,DDX11L1
4,0.0,0.000,DDX11L1
...,...,...,...
2288121,1.0,0.992,MECP2
2288122,1.0,0.993,MECP2
2288123,1.0,0.841,FLNA
2288124,1.0,0.760,ATP6AP1


In [58]:
df_half_merge.prediction.value_counts()

0.0    2287583
1.0        543
Name: prediction, dtype: int64

In [38]:
## only get pred pathogenic
df_merge_half_predpatho = df_half_merge.loc[df_half_merge ['prediction'].isin([1])]
df_merge_half_predpatho

,idx,prediction,score,SYMBOL
2283,2283,1.0,0.857,PRDM16
34043,34043,1.0,0.815,SLC6A9
71545,71545,1.0,0.999,RP5-898J17.1
97231,97231,1.0,0.862,ANP32E
98734,98734,1.0,0.886,FLG2
...,...,...,...,...
2288121,2288121,1.0,0.992,MECP2
2288122,2288122,1.0,0.993,MECP2
2288123,2288123,1.0,0.841,FLNA
2288124,2288124,1.0,0.760,ATP6AP1


## check the patho prediction by LG model , how many match genelist

In [39]:
## LG  list_INTE :  how many pathogenic prediction's 'Symbol' matched in the gene lists' 'Entity Name'

# Merging the dataframes based on the columns 'a' and 'b'
match_result_LG_INT = pd.merge(df_merge_half_predpatho, df_INTE, left_on='SYMBOL', right_on='Entity Name', how='inner')

# Printing the merge result and count of matches
match_result_LG_INT
print("Number of Matches: ", len(match_result_LG_INT))

#### count the unique values in matching:
unique_values = match_result_LG_INT['SYMBOL'].unique()
print('the number of unique matching:',len(unique_values))

Number of Matches:  312
the number of unique matching: 226


In [40]:
## LG  df_PRI :  how many pathogenic prediction's 'Symbol' matched in the gene lists' 'Entity Name'

# Merging the dataframes based on the columns 'a' and 'b'
match_result_LG_PRI = pd.merge(df_merge_half_predpatho, df_PRI, left_on='SYMBOL', right_on='Entity Name', how='inner')

# Printing the merge result and count of matches
match_result_LG_PRI
print("Number of Matches: ", len(match_result_LG_PRI))

unique_values = match_result_LG_PRI['SYMBOL'].unique()
print('the number of unique matching:',len(unique_values))

Number of Matches:  11
the number of unique matching: 8


In [41]:
## LG  df_STR :  how many pathogenic prediction's 'Symbol' matched in the gene lists' 'Entity Name'

# Merging the dataframes based on the columns 'a' and 'b'
match_result_LG_STR = pd.merge(df_merge_half_predpatho, df_STR, left_on='SYMBOL', right_on='Entity Name', how='inner')

# Printing the merge result and count of matches
match_result_LG_STR
print("Number of Matches: ", len(match_result_LG_STR))

unique_values = match_result_LG_STR['SYMBOL'].unique()
print('the number of unique matching:',len(unique_values))

Number of Matches:  58
the number of unique matching: 37
